In [34]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
import mpld3
import nltk

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rvpow\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rvpow\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [76]:
listing_data_to_load = "NO_listings.csv"
listing_data = pd.read_csv(listing_data_to_load)

review_data_to_load = "NO_reviews.csv"
review_data = pd.read_csv(review_data_to_load)

In [80]:
Price = listing_data["price"]

In [81]:
pd.to_numeric(Price)

0       300
1       100
2       125
3       115
4        50
5       123
6       130
7       105
8        99
9        91
10       80
11      102
12       75
13      125
14       85
15      100
16      245
17      130
18      107
19      159
20       99
21       45
22       95
23      120
24       89
25       75
26      200
27      210
28       95
29      115
       ... 
6602    222
6603    222
6604    222
6605    211
6606    203
6607    203
6608    203
6609    222
6610    222
6611    250
6612    185
6613    300
6614     70
6615     70
6616     70
6617    175
6618    200
6619    150
6620    250
6621    200
6622     74
6623    350
6624    150
6625    100
6626    195
6627    800
6628    464
6629     56
6630     75
6631     50
Name: price, Length: 6632, dtype: int64

In [82]:
listing_data["Price"] = Price

In [83]:
listing_data.head()

,id,listing_url,name,host_name,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,Price
0,10291,https://www.airbnb.com/rooms/10291,Spacious Cottage in Mid-City!,Jill,Navarre,29.98666,-90.10928,Cottage,Entire home/apt,4,2.0,3.0,3.0,300,2,300
1,19091,https://www.airbnb.com/rooms/19091,Fully Furnished Cozy Apartment,John,Leonidas,29.96257,-90.11877,Apartment,Entire home/apt,3,1.0,1.0,1.0,100,1,100
2,26834,https://www.airbnb.com/rooms/26834,Maison Mandeville in the Marigny,Rachel,Marigny,29.96557,-90.05531,House,Entire home/apt,2,1.0,1.0,1.0,125,4,125
3,53173,https://www.airbnb.com/rooms/53173,MARDI GRAS YA’LL HUNDREDS 5 STAR REVIEWS !!!,Karen,St. Claude,29.96483,-90.03906,House,Private room,2,2.0,1.0,1.0,115,3,115
4,53544,https://www.airbnb.com/rooms/53544,Historical Bywater Mother in Law Cottage,Lee,Bywater,29.96697,-90.03068,House,Entire home/apt,2,1.0,1.0,1.0,50,5,50


In [84]:
listing_data.dtypes

id                          int64
listing_url                object
name                       object
host_name                  object
neighbourhood_cleansed     object
latitude                  float64
longitude                 float64
property_type              object
room_type                  object
accommodates                int64
bathrooms                 float64
bedrooms                  float64
beds                      float64
price                       int64
minimum_nights              int64
Price                       int64
dtype: object

In [61]:
review_data.head()

,listing_id,comments
0,10291,We never made it but Jill called us the night ...
1,10291,Jill's place was amazing! Not only did she hel...
2,10291,This New Orleans cottage only fifteen minutes ...
3,10291,Jill's house is perfectly located. We could w...
4,10291,I was hesitant to rent Spacious Cottage becaus...


In [30]:
group_review = review_data.("listing_id")

In [12]:
stopwords = nltk.corpus.stopwords.words('english')


In [13]:
print(stopwords[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [14]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [15]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [22]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in review_data["comments"]:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

TypeError: expected string or bytes-like object

In [25]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [26]:
vocab_frame

,words
we,we
never,never
made,made
it,it
but,but
jill,jill
call,called
us,us
the,the
night,night


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

In [49]:
print(tfidf_vectorizer)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=200000, min_df=0.2,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize_and_stem at 0x0000010F09CF1F28>,
        use_idf=True, vocabulary=None)


In [48]:
tfidf_matrix = tfidf_vectorizer.fit_transform(review_data["comments"].values.astype("U"))


In [50]:
print(tfidf_matrix.shape)

(276455, 14)


In [51]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

### dist doesn't work, memory error??

# K-means clustering

In [53]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

Wall time: 4min 2s


In [58]:
from sklearn.externals import joblib

joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [95]:
price = listing_data["Price"]
id = review_data["listing_id"]

In [100]:
print(price)

0       300
1       100
2       125
3       115
4        50
5       123
6       130
7       105
8        99
9        91
10       80
11      102
12       75
13      125
14       85
15      100
16      245
17      130
18      107
19      159
20       99
21       45
22       95
23      120
24       89
25       75
26      200
27      210
28       95
29      115
       ... 
6602    222
6603    222
6604    222
6605    211
6606    203
6607    203
6608    203
6609    222
6610    222
6611    250
6612    185
6613    300
6614     70
6615     70
6616     70
6617    175
6618    200
6619    150
6620    250
6621    200
6622     74
6623    350
6624    150
6625    100
6626    195
6627    800
6628    464
6629     56
6630     75
6631     50
Name: Price, Length: 6632, dtype: int64


In [99]:
reviews = {"id": id, "price": price, "cluster": clusters}
frame = pd.DataFrame(reviews, index = [clusters], columns = ['id', 'price', 'cluster'])

In [94]:
frame

,id,price,cluster
2,NaN,NaN,2
4,NaN,NaN,4
0,NaN,NaN,0
2,NaN,NaN,2
1,NaN,NaN,1
0,NaN,NaN,0
0,NaN,NaN,0
2,NaN,NaN,2
0,NaN,NaN,0
0,NaN,NaN,0


In [91]:
frame['cluster'].value_counts()

4    26
0    25
2    21
1    16
3    12
Name: cluster, dtype: int64

In [93]:
grouped = frame['price'].groupby(frame['cluster'])

grouped.mean()

cluster
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: price, dtype: float64